In [1]:
import pandas as pd

In [2]:
# The archive was first saved as processed in this cell and then integrated with new dates.
# After the first integration, this cell became redundant, but I am keeping it only for later reference.
# Second cell will process existing complete file (dates from 1677 onwards), needed for concatentation of new data.

# Processing database with all historic earthquakes (version not available anymore).
# Downloaded from official INFP site (The National Institute for Research and Development in Earth Physics).

cols = ['DATE', 'TIME', 'LATITUDE', 'LONGITUDE', 'DEPTH', 'Mw']
df_archive = pd.read_csv('Assets/RAW_Earthquake database 984 to 30062023.csv', usecols=cols, index_col='DATE')

df_archive.index = pd.to_datetime(df_archive.index, format='mixed', errors='coerce')
df_archive = df_archive[df_archive.index.notnull()]  # Dates older than 1677 will not convert, and choosing to ignore this at this point.

df_archive['TIME'] = pd.to_datetime(df_archive['TIME'], format='%H:%M:%S.%f').dt.time
df_archive['DEPTH'] = pd.to_numeric(df_archive['DEPTH'], errors='coerce').fillna(0)

In [3]:
# Update the file as per the last processed file with the full data
df_all = pd.read_csv('Assets/Earthquake database 01011677 to 11042025.csv', index_col='DATE', parse_dates=True)

In [4]:
# Process new earthqakes downloaded from INFP site (dates later than the newest date in the df_all dataframe)
# Link: https://ph.infp.ro/seismicity/data
cols = ['time', 'lat', 'lon', 'depth', 'magnitude']
new_earthq = pd.read_csv('Assets/RAW_events 17022025-11042025.csv', usecols=cols, index_col='time', parse_dates=True)

# In the original (historic) database, TIME and DATE were separated, so keeping it the same way.
new_earthq.insert(0, 'TIME', new_earthq.index.time, True)
new_earthq.index = pd.to_datetime(new_earthq.index.date, errors='coerce')

new_earthq.index.rename('DATE', inplace=True)
new_earthq.rename(columns={
    'lat': 'LATITUDE',
    'lon': 'LONGITUDE',
    'depth': 'DEPTH',
    'magnitude': 'Mw',
}, inplace=True)

new_earthq.sort_index(ascending=True, inplace=True)

new_earthq.head()

,TIME,LATITUDE,LONGITUDE,DEPTH,Mw
DATE,,,,,
2025-02-17,19:35:59,46.4466,21.8092,1.7,3.2
2025-02-22,03:45:44,45.6202,26.5769,103.5,3.0
2025-02-23,10:47:25,45.8627,26.8296,65.0,2.2
2025-02-25,01:21:37,45.7334,26.6179,74.7,2.7
2025-02-25,01:27:29,45.7508,26.6400,74.6,2.8


In [5]:
# Concatenate existing file with the newly downloaded file
df_final = pd.concat([df_all, new_earthq])
df_final

,TIME,LATITUDE,LONGITUDE,DEPTH,Mw
DATE,,,,,
1679-08-09,01:00:00,45.7000,26.6000,110.0,7.5
1681-08-19,00:00:00,45.7000,26.6000,150.0,7.1
1681-10-16,00:00:00,45.7000,26.6000,140.0,5.4
1681-10-18,00:00:00,45.7000,26.6000,130.0,5.4
1681-12-27,04:30:00,45.7000,26.6000,110.0,5.9
...,...,...,...,...,...
2025-04-02,21:54:05,45.7451,26.7418,138.3,2.0
2025-04-04,11:50:15,45.6395,26.6108,129.0,3.2
2025-04-04,20:14:26,45.4099,26.3265,127.0,2.7


In [6]:
# Save the concatenated data by uncommenting the below line
# And by renaming the file according to the last date in your data

# df_final.to_csv('Assets/Earthquake database 01011677 to 11042025.csv')